env: cpa

# import

In [1]:
import sys
import json
from anndata import AnnData
import cpa
import scanpy as sc
import importlib
import numpy as np
from tqdm import tqdm
import pickle
import os
import shutil
import torch
import pandas as pd

sys.path.append("/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/")
import v1
from v1.utils import *
from v1.dataloader import *

Global seed set to 0


In [2]:
importlib.reload(v1)
importlib.reload(v1.utils)
importlib.reload(v1.dataloader)

<module 'v1.dataloader' from '/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/dataloader.py'>

# load K562 gwps

In [3]:
prefix = 'ReplogleWeissman2022_K562_gwps'

print('='*20, prefix)

# - init para
data_dir = '/nfs/public/lichen/data/single_cell/perturb_data/scPerturb/raw/scPerturb_rna/statistic_20240520'
pert_cell_filter = 100 # this is used to filter perts, cell number less than this will be filtered
seed = 2024 # this is the random seed
split_type = 1 # 1 for unseen perts; 0 for unseen celltypes
split_ratio = [0.7, 0.2, 0.1] # train:test:val; val is used to choose data, test is for final validation
var_num = 5000 # selecting hvg number
num_de_genes = 20 # number of de genes
bs_train = 32 # batch size of trainloader
bs_test = 32 # batch size of testloader
lr = 1e-4

tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/scPerturb'
# save_prefix = f'GEARS'
save_prefix = f'GEARS-prefix_{prefix}-pert_cell_filter_{pert_cell_filter}-\
seed_{seed}-split_type_{split_type}-var_num_{var_num}-num_de_genes_{num_de_genes}-bs_train_{bs_train}-\
bs_test_{bs_test}'

save_dir = os.path.join(tmp_dir, prefix, save_prefix)

# - load pert_data
pert_data = pickle.load(open(os.path.join(save_dir,'pert_data.pkl'), 'rb'))

==================== ReplogleWeissman2022_K562_gwps


In [ ]:
adata_ori = pert_data.adata_ori
sc.pp.normalize_per_cell(adata_ori, key_n_counts='n_counts_all')
sc.pp.log1p(adata_ori)

# 存储可以用来计算的adata_ori

In [12]:
sc.pp.normalize_per_cell(pert_data.adata_ori, key_n_counts='n_counts_all')
sc.pp.log1p(pert_data.adata_ori)

In [13]:
np.max(pert_data.adata_ori.X)

8.268049

In [4]:
# - get cell line name
common_cell_line = \
{   'A549': 'A549',
    'HEPG2': 'HepG2',
    'HT29': 'HT29',
    'MCF7': 'MCF7',
    # 'SKBR3': 'SK-BR-3',
    'SW480': 'SW480',
    'PC3': 'PC3',
    'A375': 'A375',
} # L1000 cell line : single-cell cell line

# - read adata_L1000, this is processed data
adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')
adata_L1000

AnnData object with n_obs × n_vars = 36720 × 978
    obs: 'sig_id', 'pert_id', 'pert_iname', 'pert_type', 'cell_id', 'pert_dose', 'pert_dose_unit', 'pert_idose', 'pert_time', 'pert_time_unit', 'pert_itime', 'distil_id'
    var: 'pr_gene_id', 'pr_gene_symbol', 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'

In [5]:
L1000_total_perts = []
for cell_line_bulk in common_cell_line.keys():
    cell_line_single = common_cell_line[cell_line_bulk]

    adata_L1000_sub = adata_L1000[adata_L1000.obs['cell_id']==cell_line_bulk]
    L1000_total_perts += list(np.unique(adata_L1000_sub.obs['pert_iname']))
L1000_total_perts = np.unique(L1000_total_perts)

    
import json
with open('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/utils_data/direct_change_dict.json', 'r') as f:
    direct_change_dict = json.load(f)
# - single_total_perts: the perts used in K562
gene_list = direct_change_dict['gene_list']
single_total_perts = list(direct_change_dict.keys())
single_total_perts.remove('gene_list')

common_perts = np.intersect1d(single_total_perts, L1000_total_perts)
len(common_perts)

2359

In [10]:
common_pert_groups = [i+' | K562' for i in common_perts] + ['control | K562']
adata_sub = pert_data.adata_split[pert_data.adata_split.obs['perturbation_group'].isin(common_pert_groups)]

In [11]:
adata_sub.obs['perturbation_group'].value_counts()

control | K562    75328
DUSP9 | K562       1351
RAP1GAP | K562     1270
STK38L | K562      1169
PPP2R1A | K562     1061
                  ...  
CDK6 | K562          96
HIC2 | K562          93
DDB1 | K562          70
MED15 | K562         59
AP2M1 | K562         51
Name: perturbation_group, Length: 2360, dtype: int64

In [14]:
adata_sub_ori = pert_data.adata_ori[adata_sub.obs_names].copy()
adata_sub_ori.obs = adata_sub.obs

In [17]:
adata_sub_ori

AnnData object with n_obs × n_vars = 607014 × 8248
    obs: 'batch', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'guide_id', 'percent_mito', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count', 'disease', 'cancer', 'cell_line', 'sex', 'age', 'perturbation', 'organism', 'perturbation_type', 'tissue_type', 'ncounts', 'ngenes', 'nperts', 'percent_ribo', 'perturbation_new', 'perturbation_type_new', 'nperts_new', 'celltype', 'celltype_new', 'perturbation_group', 'data_split', 'retain', 'n_genes', 'control_barcode', 'sgRNA_new', 'sgRNA_ID', 'pert_sgRNA'
    var: 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano', 'ensembl_id', 'ncounts', 'ncells'
    uns: 'log1p'

In [16]:
type(adata_sub_ori.X)

numpy.ndarray

In [18]:
import scipy.sparse

# 检查是否已经是稀疏矩阵
if not scipy.sparse.issparse(adata_sub_ori.X):
    # 如果不是稀疏矩阵，转换为稀疏矩阵
    adata_sub_ori.X = scipy.sparse.csr_matrix(adata_sub_ori.X)
    
    

In [23]:
adata_sub_ori

AnnData object with n_obs × n_vars = 607014 × 8248
    obs: 'batch', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'guide_id', 'percent_mito', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count', 'disease', 'cancer', 'cell_line', 'sex', 'age', 'perturbation', 'organism', 'perturbation_type', 'tissue_type', 'ncounts', 'ngenes', 'nperts', 'percent_ribo', 'perturbation_new', 'perturbation_type_new', 'nperts_new', 'celltype', 'celltype_new', 'perturbation_group', 'data_split', 'retain', 'n_genes', 'control_barcode', 'sgRNA_new', 'sgRNA_ID', 'pert_sgRNA'
    var: 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano', 'ensembl_id', 'ncounts', 'ncells'
    uns: 'log1p'

In [29]:
adata_sub_ori.obs = adata_sub_ori.obs[['perturbation_group', 'control_barcode']]

In [30]:
adata_sub_ori

AnnData object with n_obs × n_vars = 607014 × 8248
    obs: 'perturbation_group', 'control_barcode'
    var: 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano', 'ensembl_id', 'ncounts', 'ncells'
    uns: 'log1p'

In [31]:
adata_sub_ori.write('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/utils_data/adata_K562_sub.h5ad')

: 

# get common perts

In [ ]:
# - get cell line name
common_cell_line = \
{   'A549': 'A549',
    'HEPG2': 'HepG2',
    'HT29': 'HT29',
    'MCF7': 'MCF7',
    # 'SKBR3': 'SK-BR-3',
    'SW480': 'SW480',
    'PC3': 'PC3',
    'A375': 'A375',
} # L1000 cell line : single-cell cell line

# - read adata_L1000, this is processed data
adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')
adata_L1000

cell_line_bulk = 'A549'
cell_line_single = common_cell_line[cell_line_bulk]

In [ ]:
    
import json
with open('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/utils_data/direct_change_dict.json', 'r') as f:
    direct_change_dict = json.load(f)
# - single_total_perts: the perts used in K562
gene_list = direct_change_dict['gene_list']
single_total_perts = list(direct_change_dict.keys())
single_total_perts.remove('gene_list')

# - get common pert
adata_L1000_sub = adata_L1000[adata_L1000.obs['cell_id']==cell_line_bulk]
L1000_total_perts = np.unique(adata_L1000_sub.obs['pert_iname'])
common_perts = np.intersect1d(single_total_perts, L1000_total_perts)

In [ ]:
len(common_perts)

2184

In [ ]:
print('='*20, f'cell line is {cell_line_single}')

#===================prepare data
if cell_line_bulk in ['PC3', 'A375']:
    save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/SCP542'
    adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))
    
    # - read adata_rna_raw
    save_dir = f'/nfs/public/lichen/data/single_cell/cell_line/SCP542/process/{cell_line_bulk}'
    adata_rna_raw = sc.read(os.path.join(save_dir, f'adata.h5ad'))

else:
    save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/CNP0003658'
    adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))

    # - read adata_rna
    save_dir = f'/nfs/public/lichen/data/single_cell/cell_line/CNP0003658/process/RNA/{cell_line_single}'
    adata_rna_raw = sc.read(os.path.join(save_dir, f'adata_rna_{cell_line_single}.h5ad'))

# - consctrut corr mtx
if not isinstance(adata_rna.X, np.ndarray):
    adata_rna.X = adata_rna.X.toarray()

==================== cell line is A549


In [ ]:
# - get common var
common_var = np.intersect1d(adata_rna.var_names, direct_change_dict['gene_list'])
common_var_2 = np.intersect1d(common_var, adata_L1000.var_names)

print('common_perts num: ', len(common_perts))
print('common var of direct change and single-cell data is: ', len(common_var))
print('common var to L1000 data is: ', len(common_var_2))

common_perts num:  2184
common var of direct change and single-cell data is:  2764
common var to L1000 data is:  712


In [ ]:
pert = common_perts[0]

In [ ]:
adata_pert = pert_data.adata_split[pert_data.adata_split[pert_data.adata_split.obs['perturbation_group']==pert+' | K562'].obs_names]
adata_ctrl = pert_data.adata_split[adata_pert.obs['control_barcode']]

adata_pert = adata_ori[adata_pert.obs_names, common_var]
adata_ctrl = adata_ori[adata_ctrl.obs_names, common_var]

In [ ]:
adata_ctrl, adata_pert

(View of AnnData object with n_obs × n_vars = 182 × 2764
     obs: 'batch', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'guide_id', 'percent_mito', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count', 'disease', 'cancer', 'cell_line', 'sex', 'age', 'perturbation', 'organism', 'perturbation_type', 'tissue_type', 'ncounts', 'ngenes', 'nperts', 'percent_ribo', 'perturbation_new', 'perturbation_type_new', 'nperts_new', 'celltype', 'celltype_new', 'perturbation_group', 'data_split', 'retain', 'n_counts_all'
     var: 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano', 'ensembl_id', 'ncounts', 'ncells'
     uns: 'log1p',
 View of AnnData object with n_obs × n_vars = 182 × 2764
     obs: 'batch', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'guide_id', 'percent_mito', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count', 'disease', 'cancer', 'cell_line', 'sex', 'age', 'perturbation', 'o

In [ ]:
adata_rna_common = adata_rna[:, common_var]
adata_rna_common

View of AnnData object with n_obs × n_vars = 500 × 2764
    obs: 'n_genes', 'n_counts_all'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'

In [ ]:
# - model initial
model_params = {
    "n_latent": 64,
    "recon_loss": "nb",
    "doser_type": "linear",
    "n_hidden_encoder": 128,
    "n_layers_encoder": 2,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 2,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": False,
    "use_layer_norm_decoder": True,
    "dropout_rate_encoder": 0.0,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 6977,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 0,
    "mixup_alpha": 0.0,
    "adv_steps": None,
    "n_hidden_adv": 64,
    "n_layers_adv": 3,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 5.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": True,
    "gradient_clip_value": 1.0,
    "step_size_lr": 10,
}

In [ ]:
# - generate adata_train to input to scGen model
np_list, obs_list, pert_list, celltype_list = [], [], [], []
pert_list_2 = []
adata_list = [adata_pert, adata_rna_common, adata_ctrl, adata_rna_common]
for j, adata_ in enumerate(adata_list):
    if j in [0, 1]:
        pert_list.extend(['stimulated']*len(adata_))
    else:
        pert_list.extend(['ctrl']*len(adata_))

    if j in [0, 2]:
        celltype_list.extend(['K562']*len(adata_))
    else:
        celltype_list.extend([cell_line_bulk]*len(adata_))
    obs_list.extend([obs+f'_{j}' for obs in adata_.obs_names])
    np_list.append(adata_.X.toarray())

adata_train = AnnData(X = np.vstack(np_list))
adata_train.obs_names = obs_list
adata_train.var_names = adata_pert.var_names

adata_train.obs['condition'] = pert_list
# adata_train.obs['condition_2'] = pert_list_2
adata_train.obs['cell_type'] = celltype_list

# - transform the adata_train.X to count
adata_train.obs['cov_cond'] = adata_train.obs['cell_type'] + '_' + adata_train.obs['condition']
adata_train.X = np.exp(adata_train.X)-1

# - initial model
cpa.CPA.setup_anndata(adata_train, 
                    perturbation_key='condition',
                    control_group='ctrl',
                    #   dosage_key='dose',
                    categorical_covariate_keys=['cell_type'],
                    is_count_data=True,
                    #   deg_uns_key='rank_genes_groups_cov',
                    deg_uns_cat_key='cov_cond',
                    max_comb_len=1,
                    )

100%|██████████| 1364/1364 [00:00<00:00, 1131085.54it/s]

INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [ ]:
# - set the train and validation for cpa
# -- get total obs_names of the pert
adata_train_new = adata_train[~((adata_train.obs["cell_type"] == cell_line_bulk) &
                    (adata_train.obs["condition"] == "stimulated"))].copy()
# obs_df_split = adata_train_new.obs
obs_df_sub_idx = np.array(adata_train_new.obs.index)

np.random.seed(2024)
np.random.shuffle(obs_df_sub_idx)

# -- data split
split_point_1 = int(len(obs_df_sub_idx) * 0.9)
split_point_2 = int(len(obs_df_sub_idx) * (0.9+0.1))
train = obs_df_sub_idx[:split_point_1]
valid = obs_df_sub_idx[split_point_1:split_point_2]


adata_train.obs['split_key'] = 'ood'

# -- set the test row
adata_train.obs.loc[train,'split_key'] = 'train'
adata_train.obs.loc[valid,'split_key'] = 'valid'

# - initial the model and training   
model = cpa.CPA(adata=adata_train, 
                split_key='split_key',
                train_split='train',
                valid_split='valid',
                test_split='ood',
                **model_params,
            )

model.train(max_epochs=2000,
            use_gpu=True, 
            batch_size=500,
            plan_kwargs=trainer_params,
            early_stopping_patience=5,
            check_val_every_n_epoch=5,
            # save_path='../../datasets/',
            progress_bar_refresh_rate = 0
        )

# - predict result
model.predict(adata_train, batch_size=2048)

# - get the pred data
cat = cell_line_bulk + '_' + 'stimulated'
cat_adata = adata_train[adata_train.obs['cov_cond'] == cat].copy()
x_pred = cat_adata.obsm['CPA_pred']
x_pred = np.log1p(x_pred)

Global seed set to 6977


100%|██████████| 2/2 [00:00<00:00, 708.92it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

Epoch 00004: cpa_metric reached. Module best state updated.

Epoch 00009: cpa_metric reached. Module best state updated.



disnt_basal = 1.692965367965368
disnt_after = 1.7727272727272727
val_r2_mean = 0.3614012598991394
val_r2_var = -2.6723037560780845



Epoch 00014: cpa_metric reached. Module best state updated.

Epoch 00019: cpa_metric reached. Module best state updated.



disnt_basal = 1.6858585858585857
disnt_after = 1.7727272727272727
val_r2_mean = 0.7982175350189209
val_r2_var = -2.674081643422445


...


Epoch 00084: cpa_metric reached. Module best state updated.

Epoch 00089: cpa_metric reached. Module best state updated.



disnt_basal = 1.7011202761202762
disnt_after = 1.7254231504231505
val_r2_mean = 0.9396191239356995
val_r2_var = -2.3239022890726724



Epoch 00094: cpa_metric reached. Module best state updated.

Epoch 00099: cpa_metric reached. Module best state updated.



disnt_basal = 1.5094290784290785
disnt_after = 1.7252252252252251
val_r2_mean = 0.9363454778989156
val_r2_var = -2.2833077112833657

disnt_basal = 1.6963924963924963
disnt_after = 1.7361111111111112
val_r2_mean = 0.9404150446256002
val_r2_var = -2.2283284664154053

disnt_basal = 1.697150072150072
disnt_after = 1.7291837291837293
val_r2_mean = 0.933383027712504
val_r2_var = -2.1212982336680093


100%|██████████| 1/1 [00:00<00:00, 20.75it/s]


In [ ]:
x_pred

array([[0.59670675, 0.57625264, 0.918021  , ..., 1.582566  , 1.4443177 ,
        0.34309846],
       [0.7520548 , 0.5868137 , 1.0911624 , ..., 1.6446931 , 1.3728073 ,
        0.3184219 ],
       [0.66804856, 0.58108974, 0.913697  , ..., 1.6078756 , 1.4655265 ,
        0.32640746],
       ...,
       [0.6869034 , 0.546764  , 1.0182772 , ..., 1.6756613 , 1.4602138 ,
        0.35031497],
       [0.6638175 , 0.57233644, 1.0785822 , ..., 1.6767553 , 1.427946  ,
        0.30222225],
       [0.6788921 , 0.57769287, 0.99439704, ..., 1.9601779 , 1.5734761 ,
        0.4031645 ]], dtype=float32)

In [ ]:
adata_rna_common.X

ArrayView([[0.        , 0.7986983 , 0.7986983 , ..., 0.7986983 ,
            0.7986983 , 0.7986983 ],
           [0.5719357 , 0.9334965 , 2.072573  , ..., 1.9704001 ,
            0.5719357 , 0.        ],
           [0.        , 0.8364108 , 2.1802375 , ..., 1.2854061 ,
            1.0858992 , 0.50321716],
           ...,
           [0.        , 1.5602019 , 1.7939309 , ..., 1.2546245 ,
            0.        , 0.        ],
           [0.        , 1.1712079 , 1.1712079 , ..., 2.038315  ,
            0.90983987, 0.        ],
           [0.9632273 , 0.59336513, 1.7682205 , ..., 1.4446287 ,
            1.7682205 , 0.59336513]], dtype=float32)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# ?model.train

In [ ]:
common_idx = [direct_change_dict['gene_list'].index(gene) for i, gene in enumerate(common_var)]

# 并行加速搞不通，CPA需要torch，有点复杂

In [ ]:
def cpa_single_pert(pert):
    
    # - get adata_pert and adata_ctrl
    adata_pert = pert_data.adata_split[pert_data.adata_split[pert_data.adata_split.obs['perturbation_group']==pert+' | K562'].obs_names]
    adata_ctrl = pert_data.adata_split[adata_pert.obs['control_barcode']]

    adata_pert = adata_ori[adata_pert.obs_names, common_var]
    adata_ctrl = adata_ori[adata_ctrl.obs_names, common_var]


    # - get adata_rna_common
    adata_rna_common = adata_rna[:, common_var]

    # - generate adata_train to input to scGen model
    np_list, obs_list, pert_list, celltype_list = [], [], [], []
    pert_list_2 = []
    adata_list = [adata_pert, adata_rna_common, adata_ctrl, adata_rna_common]
    for j, adata_ in enumerate(adata_list):
        if j in [0, 1]:
            pert_list.extend(['stimulated']*len(adata_))
        else:
            pert_list.extend(['ctrl']*len(adata_))

        if j in [0, 2]:
            celltype_list.extend(['K562']*len(adata_))
        else:
            celltype_list.extend([cell_line_bulk]*len(adata_))
        obs_list.extend([obs+f'_{j}' for obs in adata_.obs_names])
        np_list.append(adata_.X.toarray())

    adata_train = AnnData(X = np.vstack(np_list))
    adata_train.obs_names = obs_list
    adata_train.var_names = adata_pert.var_names

    adata_train.obs['condition'] = pert_list
    # adata_train.obs['condition_2'] = pert_list_2
    adata_train.obs['cell_type'] = celltype_list

    # - transform the adata_train.X to count
    adata_train.obs['cov_cond'] = adata_train.obs['cell_type'] + '_' + adata_train.obs['condition']
    adata_train.X = np.exp(adata_train.X)-1

    # - initial model
    cpa.CPA.setup_anndata(adata_train, 
                        perturbation_key='condition',
                        control_group='ctrl',
                        #   dosage_key='dose',
                        categorical_covariate_keys=['cell_type'],
                        is_count_data=True,
                        #   deg_uns_key='rank_genes_groups_cov',
                        deg_uns_cat_key='cov_cond',
                        max_comb_len=1,
                        )

    # - set the train and validation for cpa
    # -- get total obs_names of the pert
    adata_train_new = adata_train[~((adata_train.obs["cell_type"] == cell_line_bulk) &
                        (adata_train.obs["condition"] == "stimulated"))].copy()
    # obs_df_split = adata_train_new.obs
    obs_df_sub_idx = np.array(adata_train_new.obs.index)

    np.random.seed(2024)
    np.random.shuffle(obs_df_sub_idx)

    # -- data split
    split_point_1 = int(len(obs_df_sub_idx) * 0.9)
    split_point_2 = int(len(obs_df_sub_idx) * (0.9+0.1))
    train = obs_df_sub_idx[:split_point_1]
    valid = obs_df_sub_idx[split_point_1:split_point_2]


    adata_train.obs['split_key'] = 'ood'

    # -- set the test row
    adata_train.obs.loc[train,'split_key'] = 'train'
    adata_train.obs.loc[valid,'split_key'] = 'valid'

    # - initial the model and training   
    model = cpa.CPA(adata=adata_train, 
                    split_key='split_key',
                    train_split='train',
                    valid_split='valid',
                    test_split='ood',
                    **model_params,
                )

    model.train(max_epochs=2000,
                use_gpu=True, 
                batch_size=500,
                plan_kwargs=trainer_params,
                early_stopping_patience=5,
                check_val_every_n_epoch=5,
                # save_path='../../datasets/',
                progress_bar_refresh_rate = 0
            )

    # - predict result
    model.predict(adata_train, batch_size=2048)

    # - get the pred data
    cat = cell_line_bulk + '_' + 'stimulated'
    cat_adata = adata_train[adata_train.obs['cov_cond'] == cat].copy()
    x_pred = cat_adata.obsm['CPA_pred']
    x_pred = np.log1p(x_pred)


    # - get pert_gene_rank_dict
    adata_ctrl = adata_rna_common.copy()
    adata_pert = adata_ctrl.copy()
    adata_pert.X = x_pred
    adata_pert.obs_names = [i+f'_{pert}' for i in adata_pert.obs_names]

    adata_ctrl.obs['batch'] = 'ctrl'
    adata_pert.obs['batch'] = 'pert'

    import anndata as ad
    adata_concat = ad.concat([adata_ctrl, adata_pert])

    # - cal de genes
    rankby_abs = False

    sc.tl.rank_genes_groups(
        adata_concat,
        groupby='batch',
        reference='ctrl',
        rankby_abs=rankby_abs,
        n_genes=len(adata_concat.var),
        use_raw=False,
        method = 'wilcoxon'
    )
    de_genes = pd.DataFrame(adata_concat.uns['rank_genes_groups']['names'])
    pvals = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals'])
    pvals_adj = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals_adj'])
    scores = pd.DataFrame(adata_concat.uns['rank_genes_groups']['scores'])
    logfoldchanges = pd.DataFrame(adata_concat.uns['rank_genes_groups']['logfoldchanges'])

    # - get gene_score
    gene_score = pd.DataFrame({'gene':list(de_genes['pert']),
                                'z-score':list(scores['pert'])})

    return (list(de_genes['pert']), list(scores['pert']))
    
pert_gene_rank_dict = {}
for pert in tqdm(common_perts, desc='cpa_single_pert'):
    pert_gene_rank_dict[pert] = cpa_single_pert(pert)


cpa_single_pert:   0%|          | 0/2184 [00:00<?, ?it/s]

100%|██████████| 1364/1364 [00:00<00:00, 1085378.61it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


Global seed set to 6977
100%|██████████| 2/2 [00:00<00:00, 950.98it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

Epoch 00004: cpa_metric reached. Module best state updated.

Epoch 00009: cpa_metric reached. Module best state updated.



disnt_basal = 1.692965367965368
disnt_after = 1.7727272727272727
val_r2_mean = 0.3614012598991394
val_r2_var = -2.6723037560780845


...


Epoch 00024: cpa_metric reached. Module best state updated.

Epoch 00029: cpa_metric reached. Module best state updated.



disnt_basal = 1.6174863387978142
disnt_after = 1.5806719287593607
val_r2_mean = 0.9061456521352133
val_r2_var = -2.384357213973999



Epoch 00039: cpa_metric reached. Module best state updated.



disnt_basal = 1.6174863387978142
disnt_after = 1.5555555555555556
val_r2_mean = 0.924484392007192
val_r2_var = -2.3337789376576743


cpa_single_pert:   2%|▏         | 49/2184 [13:41<9:56:39, 16.77s/it] 

KeyboardInterrupt



# 尝试看看cell_line_bulk的并行能否成功

In [52]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, auc
from scipy.spatial.distance import cdist
import concurrent.futures
import json
import anndata as ad


Process SpawnProcess-436:
Traceback (most recent call last):
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'process_cell_line' on <module '__main__' (built-in)>
Process SpawnProcess-437:
Traceback (most recent call last):
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/process.py", line 108, in r

Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.


In [57]:


    
    


# 定义处理每个 cell_line_single 的函数
def process_cell_line(cell_line_bulk, cell_line_single, common_cell_line, adata_L1000, gpu_id):
    print('=' * 20, f'cell line is {cell_line_single}')
    
    torch.cuda.set_device(gpu_id)  # 设置每个进程使用不同的 GPU

    #===================prepare data
    if cell_line_bulk in ['PC3', 'A375']:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/SCP542'
        adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))
        
        # - read adata_rna_raw
        save_dir = f'/nfs/public/lichen/data/single_cell/cell_line/SCP542/process/{cell_line_bulk}'
        adata_rna_raw = sc.read(os.path.join(save_dir, f'adata.h5ad'))
    
    else:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/CNP0003658'
        adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))

        # - read adata_rna
        save_dir = f'/nfs/public/lichen/data/single_cell/cell_line/CNP0003658/process/RNA/{cell_line_single}'
        adata_rna_raw = sc.read(os.path.join(save_dir, f'adata_rna_{cell_line_single}.h5ad'))

    # - consctrut corr mtx
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    corr_mtx = np.corrcoef(adata_rna.X.T)
    
    # - get var_names
    var_names = list(adata_rna.var_names)
    
    # - get common pert
    adata_L1000_sub = adata_L1000[adata_L1000.obs['cell_id']==cell_line_bulk]
    L1000_total_perts = np.unique(adata_L1000_sub.obs['pert_iname'])
    common_perts = np.intersect1d(adata_rna.var_names, L1000_total_perts)

    
    
    print('common_perts num: ', len(common_perts))
    print('common var to L1000 data is: ', len(np.intersect1d(var_names, adata_L1000.var_names)))

    
    adata_pert_list = []
    pert_gene_rank_dict = {}
    for pert in tqdm(common_perts, desc='cpa_single_pert'):
        pert_gene_rank_dict[pert] = cpa_single_pert(pert)
        
        
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/zero_shot/result'
    save_prefix = f'CPA/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(save_dir, save_prefix), exist_ok=True)

    import json
    with open(os.path.join(save_dir, save_prefix, 'pert_gene_rank_dict.json'), 'w') as f:
        json.dump(pert_gene_rank_dict, f)


In [70]:
import cpa

In [68]:
sys.path.append("/data1/lichen/code/single_cell_perturbation/scPerturb_202410/zero_shot/method/CPA/")



In [83]:
import importlib
importlib.reload(cpa_func_lc)

<module 'cpa_func_lc' from '/data1/lichen/code/single_cell_perturbation/scPerturb_202410/zero_shot/method/CPA/cpa_func_lc.py'>

In [84]:
import cpa_func_lc
from cpa_func_lc import process_cell_line

In [90]:

import multiprocessing
from multiprocessing import Manager

# 主函数
if __name__ == "__main__":
    # - get cell line name
    common_cell_line = \
    {   'A549': 'A549',
        'HEPG2': 'HepG2',
        'HT29': 'HT29',
        'MCF7': 'MCF7',
        # 'SKBR3': 'SK-BR-3',
        'SW480': 'SW480',
        'PC3': 'PC3',
        'A375': 'A375',
    } # L1000 cell line : single-cell cell line
    
    multiprocessing.set_start_method('spawn', force=True)

    # - read adata_L1000, this is processed data
    adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')

    

    # 使用并行执行
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(process_cell_line, adata_ori, cell_line_bulk, common_cell_line[cell_line_bulk], adata_L1000)
            for cell_line_bulk in common_cell_line.keys()
        ]
        
        # 等待所有任务完成
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # 获取每个任务的结果，如果有异常，将在此处抛出
            except Exception as e:
                print(f"Error: {e}")


Global seed set to 0


==================== cell line is A549
Error: name 'pert_data' is not defined
common_perts num:  3096
common var to L1000 data is:  933
AARS
============ (1989578, 8248)


cpa_single_pert:   0%|          | 0/3096 [00:00<?, ?it/s]


In [50]:
import multiprocessing
multiprocessing.set_start_method('spawn', force=True)

In [ ]:
multiprocessing.get_start_method()

'spawn'

# cell_line_bulk并行-torch multiprocessing

In [56]:
import torch.multiprocessing as mp

# 主函数
if __name__ == "__main__":
    # - get cell line name
    common_cell_line = {
        'A549': 'A549',
        'HEPG2': 'HepG2',
        'HT29': 'HT29',
        'MCF7': 'MCF7',
        'SW480': 'SW480',
        'PC3': 'PC3',
        'A375': 'A375',
    } # L1000 cell line : single-cell cell line

    # 设置多进程启动方式为 'spawn'
    mp.set_start_method('spawn', force=True)

    # - read adata_L1000, this is processed data
    adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')

    # 获取 GPU 数量，并为每个进程分配一个 GPU
    num_gpus = torch.cuda.device_count()

    # 使用 torch.multiprocessing 进行并行处理
    processes = []
    for i, cell_line_bulk in enumerate(common_cell_line.keys()):
        gpu_id = i % num_gpus  # 为每个进程分配不同的 GPU
        cell_line_single = common_cell_line[cell_line_bulk]
        p = mp.Process(target=process_cell_line, args=(cell_line_bulk, cell_line_single, common_cell_line, adata_L1000, gpu_id))
        p.start()
        processes.append(p)

    # 等待所有进程完成
    for p in processes:
        p.join()

    print("All processes completed.")

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'process_cell_line' on <module '__main__' (built-in)>


KeyboardInterrupt: 

In [61]:
import torch.multiprocessing as mp
import scanpy as sc
import dill as pickle  # 使用 dill 代替 pickle


# 必须使用 dill 来序列化多进程函数
if __name__ == "__main__":
    # mp.set_start_method('spawn', force=True)
    # mp.set_executable(mp.get_executable())

    # - get cell line name
    common_cell_line = {
        'A549': 'A549',
        'HEPG2': 'HepG2',
        'HT29': 'HT29',
        'MCF7': 'MCF7',
        'SW480': 'SW480',
        'PC3': 'PC3',
        'A375': 'A375',
    }

    # - read adata_L1000, this is processed data
    adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')

    # 获取 GPU 数量，并为每个进程分配一个 GPU
    num_gpus = torch.cuda.device_count()

    # 使用 torch.multiprocessing 进行并行处理
    processes = []
    for i, cell_line_bulk in enumerate(common_cell_line.keys()):
        gpu_id = i % num_gpus  # 为每个进程分配不同的 GPU
        cell_line_single = common_cell_line[cell_line_bulk]
        p = mp.Process(target=process_cell_line, args=(cell_line_bulk, cell_line_single, common_cell_line, adata_L1000, gpu_id))
        p.start()
        processes.append(p)

    # 等待所有进程完成
    for p in processes:
        p.join()

    print("All processes completed.")


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/data1/lichen/anaconda3/envs/cpa/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'process_cell_line' on <module '__main__' (built-in)>


KeyboardInterrupt: 

# gpt改写并行版本

In [ ]:
import concurrent.futures
import pandas as pd
import numpy as np
from anndata import AnnData
import scanpy as sc

# 改进为返回值形式以便收集结果
def cpa_single_pert(pert, gpu_id = 0):
    
    # 设置 GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    print(f"Starting training for {pert} on GPU {gpu_id}")
    
    print('='*20, pert)
    global pert_data, adata_ori, adata_rna, common_var, cell_line_bulk, model_params, trainer_params

    # - get adata_pert and adata_ctrl
    adata_pert = pert_data.adata_split[pert_data.adata_split[pert_data.adata_split.obs['perturbation_group']==pert+' | K562'].obs_names]
    adata_ctrl = pert_data.adata_split[adata_pert.obs['control_barcode']]

    adata_pert = adata_ori[adata_pert.obs_names, common_var]
    adata_ctrl = adata_ori[adata_ctrl.obs_names, common_var]

    # - get adata_rna_common
    adata_rna_common = adata_rna[:, common_var]

    # - generate adata_train to input to scGen model
    np_list, obs_list, pert_list, celltype_list = [], [], [], []
    adata_list = [adata_pert, adata_rna_common, adata_ctrl, adata_rna_common]
    for j, adata_ in enumerate(adata_list):
        if j in [0, 1]:
            pert_list.extend(['stimulated']*len(adata_))
        else:
            pert_list.extend(['ctrl']*len(adata_))

        if j in [0, 2]:
            celltype_list.extend(['K562']*len(adata_))
        else:
            celltype_list.extend([cell_line_bulk]*len(adata_))
        obs_list.extend([obs+f'_{j}' for obs in adata_.obs_names])
        np_list.append(adata_.X.toarray())

    adata_train = AnnData(X=np.vstack(np_list))
    adata_train.obs_names = obs_list
    adata_train.var_names = adata_pert.var_names

    adata_train.obs['condition'] = pert_list
    adata_train.obs['cell_type'] = celltype_list

    # - transform the adata_train.X to count
    adata_train.obs['cov_cond'] = adata_train.obs['cell_type'] + '_' + adata_train.obs['condition']
    adata_train.X = np.exp(adata_train.X) - 1

    # - initialize model
    cpa.CPA.setup_anndata(adata_train, 
                          perturbation_key='condition',
                          control_group='ctrl',
                          categorical_covariate_keys=['cell_type'],
                          is_count_data=True,
                          deg_uns_cat_key='cov_cond',
                          max_comb_len=1)
    
    print('adata_train.shape: ', adata_train.shape)

    adata_train_new = adata_train[~((adata_train.obs["cell_type"] == cell_line_bulk) &
                                    (adata_train.obs["condition"] == "stimulated"))].copy()

    obs_df_sub_idx = np.array(adata_train_new.obs.index)
    np.random.seed(2024)
    np.random.shuffle(obs_df_sub_idx)

    split_point_1 = int(len(obs_df_sub_idx) * 0.9)
    split_point_2 = int(len(obs_df_sub_idx) * (0.9 + 0.1))
    train = obs_df_sub_idx[:split_point_1]
    valid = obs_df_sub_idx[split_point_1:split_point_2]

    adata_train.obs['split_key'] = 'ood'
    adata_train.obs.loc[train, 'split_key'] = 'train'
    adata_train.obs.loc[valid, 'split_key'] = 'valid'
    
    print('start training......')

    # - train the model
    model = cpa.CPA(adata=adata_train, 
                    split_key='split_key',
                    train_split='train',
                    valid_split='valid',
                    test_split='ood',
                    **model_params)

    model.train(max_epochs=2000,
                use_gpu=True, 
                batch_size=500,
                plan_kwargs=trainer_params,
                early_stopping_patience=5,
                check_val_every_n_epoch=5,
                progress_bar_refresh_rate=0)
    
    print('finish training')

    model.predict(adata_train, batch_size=2048)

    # - get the predicted data
    cat = cell_line_bulk + '_' + 'stimulated'
    cat_adata = adata_train[adata_train.obs['cov_cond'] == cat].copy()
    x_pred = cat_adata.obsm['CPA_pred']
    x_pred = np.log1p(x_pred)

    adata_ctrl = adata_rna.copy()
    adata_pert = adata_ctrl.copy()
    adata_pert.X = x_pred
    adata_pert.obs_names = [i + f'_{pert}' for i in adata_pert.obs_names]

    adata_ctrl.obs['batch'] = 'ctrl'
    adata_pert.obs['batch'] = 'pert'

    adata_concat = sc.concat([adata_ctrl, adata_pert])

    # - calculate DE genes
    sc.tl.rank_genes_groups(
        adata_concat,
        groupby='batch',
        reference='ctrl',
        rankby_abs=False,
        n_genes=len(adata_concat.var),
        use_raw=False,
        method='wilcoxon'
    )
    
    de_genes = pd.DataFrame(adata_concat.uns['rank_genes_groups']['names'])
    scores = pd.DataFrame(adata_concat.uns['rank_genes_groups']['scores'])

    # - return gene score for the current perturbation
    return pert, (list(de_genes['pert']), list(scores['pert']))



In [ ]:
# - concurrent使用gpu

if __name__ == "__main__":
    gpu_ids = [0, 1, 2, 3]  # 假设有 4 张 GPU
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=len(gpu_ids)) as executor:
        futures = {executor.submit(cpa_single_pert, pert, gpu_ids[i % len(gpu_ids)]): pert for i, pert in enumerate(common_perts[0:4])}
        
        for future in concurrent.futures.as_completed(futures):
            pert, result = future.result()
            print(f"Task for {pert} completed with result: {result}")

Starting training for ABCB7 on GPU 2Starting training for ABCB6 on GPU 1Starting training for AATF on GPU 0
Starting training for ABCC1 on GPU 3

 ============================================================ ABCB7  ABCB6
AATFABCC1





  0%|          | 0/1488 [00:00<?, ?it/s]

100%|██████████| 1402/1402 [00:00<00:00, 1088763.97it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
adata_train.shape:  (1470, 2764)adata_train.shape: 
 (1488, 2764)adata_train.shape: 
 (1402, 2764)
start training......
start training......start training......



Global seed set to 6977


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


Global seed set to 6977
Global seed set to 6977


adata_train.shape:  (1364, 2764)
start training......


Global seed set to 6977
100%|██████████| 2/2 [00:00<00:00, 934.77it/s]


RuntimeError: CUDA error: initialization error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import torch.multiprocessing as mp
import os

if __name__ == "__main__":
    gpu_ids = [0, 1, 2, 3]  # 假设有 4 张 GPU
    
    # 使用 torch.multiprocessing
    with mp.Pool(processes=len(gpu_ids)) as pool:
        results = [pool.apply_async(cpa_single_pert, args=(pert, gpu_ids[i % len(gpu_ids)])) for i, pert in enumerate(common_perts[0:4])]
        pool.close()
        pool.join()

        for result in results:
            print(result.get())

Starting training for ABCB6 on GPU 1
Starting training for ABCB7 on GPU 2Starting training for AATF on GPU 0====================

 ====================ABCB6====================
  Starting training for ABCC1 on GPU 3AATFABCB7


==================== ABCC1




  0%|          | 0/1402 [00:00<?, ?it/s]

100%|██████████| 1364/1364 [00:00<00:00, 706823.65it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
adata_train.shape: adata_train.shape: adata_train.shape:    (1364, 2764)(1470, 2764)(1488, 2764)


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                              

Global seed set to 6977
Global seed set to 6977
Global seed set to 6977


start training......


Global seed set to 6977
100%|██████████| 2/2 [00:00<00:00, 1058.63it/s]


RuntimeError: CUDA error: initialization error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:

# 使用并行处理
pert_gene_rank_dict = {}

# 并行执行每个 perturbation
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = {executor.submit(cpa_single_pert, pert): pert for pert in common_perts[0:5]}

    # 使用 tqdm 显示进度条
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(common_perts[0:5])):
        pert, gene_scores = future.result()
        pert_gene_rank_dict[pert] = gene_scores


In [ ]:
import multiprocessing

def collect_result(result):
    pert, data = result
    pert_gene_rank_dict[pert] = data
    
if __name__ == "__main__":
    manager = multiprocessing.Manager()
    pert_gene_rank_dict = manager.dict()  # 用 Manager 使其在进程之间共享
    
    pool = multiprocessing.Pool(processes=5)
    # pool = multiprocessing.cpu_count()
    # common_perts = [...]  # 你的 perturbations 列表
    
    # 使用 pool 并行执行
    for pert in common_perts[0:5]:
        pool.apply_async(cpa_single_pert, args=(pert,), callback=collect_result)

    pool.close()
    pool.join()  # 等待所有任务完成

========================================  ====================AATF====================ABCB6 

==================== ABCB7 ABCC1
ABCC10



100%|██████████| 1488/1488 [00:00<00:00, 323257.08it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
adata_train.shape:  INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
(1262, 2764)
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column n

Global seed set to 6977


start training......start training......
start training......start training......




Global seed set to 6977
Global seed set to 6977
Global seed set to 6977
Global seed set to 6977
100%|██████████| 2/2 [00:00<00:00, 570.19it/s]


In [ ]:
print(pert_gene_rank_dict)

{}
